In [ ]:
import sys
sys.path.append("..")
from f16lib.systems import F16AcasShield, F16Shield
import csaf


import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import random

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf = F16AcasShield()
my_conf.check()
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

In [ ]:
own_state = (807, 0.0389, 0.0, 0.0, 0.0389, 3.141592653589793, 0.0, 0.0, 
                        0.0, 25000, 0.0, 1000.0, 9.0567)
other_state = (807, 0.0389, 0.0, 0.0, 0.0389, 0.0, 0.0, 0.0, 0.0, 
                                 0, 0, 1000.0, 9.0567)


class F16AcasShieldEnv(csaf.SystemEnv):
    system_type = F16AcasShield
    agents = ["predictor"]
    
    
def initialize_env(sys, own_state, other_state):
    # set the own and intruder state
    sys.set_state('plant', other_state)
    sys.set_state('intruder_plant', own_state)
    
    
def get_random_states():
    init = list(other_state)
    rad = 5000
    init[10] += random.random() * 2*rad - rad
    y_rad = 200
    init[9] += random.random() * 2*y_rad - y_rad
    return tuple(init), own_state
    

In [ ]:
env = F16AcasShield()
initialize_env(env, *get_random_states())

In [ ]:
#env = F16AcasShieldEnv()

def task():
    env = F16AcasShield()
    initialize_env(env, *get_random_states())
    return env.simulate_tspan((0.0, 20.0))

In [ ]:
from joblib import Parallel, delayed
results = Parallel(n_jobs=16)(delayed(task)() for _ in range(1000))

In [ ]:
states = ['clear', 'weak-left', 'weak-right', 'strong-left', 'strong-right']#'strong-left', 'weak-left', 'clear', 'weak-right', 'strong-right']
colors = ['grey', 'skyblue', 'lightcoral', 'deepskyblue', 'firebrick']

figure = plt.figure(figsize=(8, 8))

for r in results[0:100]:
    s = np.array(r['autopilot'].states).flatten()
    p = np.array(r['plant'].states)
    p = p[:-10*20:10*20] 
    s = s[::20]
    for si, ci in zip(states, colors):
        transitions = s[1:] == si #[s1 == si for s0, s1 in zip(s[:-1], s[1:])]#[s0 != s1 and s1 == si for s0, s1 in zip(s[:-1], s[1:])]
        y, x= p[:, 9:11].T
        plt.scatter(x[transitions], y[transitions], marker='x', color=ci, s=20)
        plt.scatter(x[0], y[0], marker='o', color=ci, s=20)
        plt.scatter(x[-1], y[-1], marker='o', color='k', s=20)
        
plt.grid()
plt.axis("equal")
plt.title("Decision Points")
plt.tight_layout()
plt.xlabel("East / West Position (ft)")
plt.ylabel("North / South Position (ft)")
plt.savefig("decision_pts.pdf")
plt.show()
